# NFL Fantasy Football: Data Preprocessing

The overall goal of this notebook is to import data collected from https://github.com/amcheste/fball_data_collection and perform data preprocessing. This includes removing data that is not needed for the scope of this project, setting null values to zero and creating a data frames that can be used for data analysis and feature engineering.

In [18]:
import pandas as pd
import requests
import os

## Download Data
Due to the volume of data collected by https://github.com/amcheste/fball_data_collection, the output csv files are considered large files for git.  Therefore, various output csv files are stored in an [Object Storage](https://docs.oracle.com/en-us/iaas/Content/Object/Concepts/objectstorageoverview.htm) bucket in [Oracle Cloud Infrastructure (OCI)](https://docs.oracle.com/en-us/iaas/Content/GSG/Concepts/baremetalintro.htm).

Since the collected data does not include any sensitive data and is publicly available on the internet I created a [Pre-Authenticated Request (PAR)](https://docs.oracle.com/en-us/iaas/Content/Object/Tasks/usingpreauthenticatedrequests.htm), allowing non-authenticated users to download the files stored in the `DSC-412` bucket which is storing the various csv files containing data collected from ESPN's APIs.  This [PAR URL](https://objectstorage.us-ashburn-1.oraclecloud.com/p/gGzdsEKSIArLMAV1cP7SUkd6jSGF-P5wFn5kENCtQaABjvsLJkgJZ_vPi-27a8NL/n/id8zuxg6euyj/b/DSC-412/o/) will be valid for the duration of this class.

The downloaded csv files will be stored in directories under the `tmp` directory in this project.  If the `tmp` directory does not exist it will be created.  It should also be noted that the `tmp` directory is included in the `.gitignore` to ensure these large files are not checked into the git repository.  

The raw csv files will be stored under `tmp/00` with preprocessed data stored in `tmp/01` that will be used for data analysis and feature engineering in the `02_feature_engineering.ipnb` notebook.

In [19]:
#
# Check to see if the tmp dirs exist in this project.  We will need them to store data between notebooks.
if not os.path.isdir('../tmp'):
    os.mkdir('../tmp')
if not os.path.isdir('../tmp/00'):
    os.mkdir('../tmp/00')
if not os.path.isdir('../tmp/01'):
    os.mkdir('../tmp/01')
if not os.path.isdir('../tmp/02'):
    os.mkdir('../tmp/02')

#
# Pre-Authenticated Request URL that can be used to download the collected data.
BUCKET_URL = 'https://objectstorage.us-ashburn-1.oraclecloud.com/p/gGzdsEKSIArLMAV1cP7SUkd6jSGF-P5wFn5kENCtQaABjvsLJkgJZ_vPi-27a8NL/n/id8zuxg6euyj/b/DSC-412/o/'   

#
# Import positions raw data
ret = requests.get(BUCKET_URL)
if ret.status_code != 200:
    print("ERROR: Failed to fetch data file paths")
    print(f"ERROR: {ret.json()}")
    exit(255)
    
#
# Loop through the list of objects in the bucket.  Each object is a csv file of collected data from ESPN and Yahoo.
for object in ret.json()['objects']:
    ret = requests.get(f'{BUCKET_URL}{object['name']}')
    if ret.status_code != 200:
        print(f"ERROR: Failed to fetch {object['name']}")
        print(f"ERROR: {ret.json()}")
        exit(255)
    #
    # Save a local copy of the CSV in the tmp/00 directory
    file = open(f'../tmp/00/{object['name']}', "w")
    file.write(ret.text)
    file.close()

## NFL Positions

Our first data file contains details regarding positions.  This includes the ESPN position ID that will be refrenced through the remainder of the data.  We can use this to help filter positions we care about.  For the scope of this project we will focus on the Quarterback position.

In [20]:
#
# Import positions raw data
positions_df = pd.read_csv('../tmp/00/positions.csv')

#
# remove url column
positions_df = positions_df.drop(columns=['url'])
positions_df.set_index('id', inplace=True)

#
# Select the positions we care about and remove the remaining from our fantasy positions dataframe
fantasy_positions = ['QB']
fantasy_positions_df = positions_df[positions_df['abbreviation'].isin(fantasy_positions)]

#
# Print out the dataframe to validate and get the QB ID.
fantasy_positions_df

,name,abbreviation
id,,
8,Quarterback,QB


## NFL Players
Next we will want to import past and current stats for players.  This data was collected from ESPN and there are multiple categories of stats including general, passing, and rushing.  We will want to import all of these and create a single dataframe that includes all of these stats.


In [21]:
#
# Read in the top level player data
players_df = pd.read_csv('../tmp/00/players.csv')

#
# Remove columns that are not needed
players_df = players_df.drop(columns=['url', 'stats_log'])
players_df.set_index('id', inplace=True)

#
# If a value is NaN set it to 0
players_df.fillna(0, inplace=True)

#
# Read in general player stats
player_stats_general = pd.read_csv('../tmp/00/player_general_stats.csv')
#
# Remove columns that are not needed.  We do not need a uuid for each stats and the net_total_yards and net_yards_per_game do not have any values set.
player_stats_general = player_stats_general.drop(columns=['id', 'net_total_yards', 'net_yards_per_game'])

#
# If a value is NaN set it to 0
player_stats_general.fillna(0, inplace=True)

#
# Perform an inner join based on player ID to merge these the player stats in a single dataframe.
result = pd.merge(players_df, player_stats_general, left_on='id', right_on='player_id', how='inner')

#
# Read in the passing stats for each player.
player_stats_passing = pd.read_csv('../tmp/00/player_passing_stats.csv')

#
# We do not need a uuid for each stat entry
player_stats_passing = player_stats_passing.drop(columns=['id'])

#
# If a value is NaN set it to 0
player_stats_passing.fillna(0, inplace=True)

#
# Merge the stats in a single dataframe based on player ID
result = pd.merge(result, player_stats_passing, on='player_id', how='inner')

#
# Read in rushing stats for QBs that are mobile.  We will follow the same pattern as above.
player_stats_rushing = pd.read_csv('../tmp/00/player_rushing_stats.csv')
player_stats_rushing = player_stats_rushing.drop(columns=['id'])
player_stats_rushing.fillna(0, inplace=True)

result = pd.merge(result, player_stats_rushing, on='player_id', how='inner')


#
# Now that we have a single dataframe lets remove non-active players and players who are not QBs
index_to_drop = result[result['active'] != True].index
result = result.drop(index_to_drop)
index_to_drop = result[result['status'] != 'Active'].index
result = result.drop(index_to_drop)

#
# Remove non QB positions.  
# TODO: Remove magic number and read this programmatically.
index_to_drop = result[result['position'] != 8].index
result = result.drop(index_to_drop)

#
# Save this dataframe for future use.
result.to_csv('../tmp/01/player_stats.csv')



## Player Fantasy Points
Next we will want to import the data set collected from Yahoo that contains per game stats from the 2024 season along with the calculated fantasy football points based on their default scoring scheme.

In [22]:
#
# Read in fantasy points
fpts_df = pd.read_csv('../tmp/00/player_points.csv')

#
# Remove anyone who did not score any points since they most likely did not play
index_to_drop = fpts_df[fpts_df['points'] == 0.00].index
fpts_df = fpts_df.drop(index_to_drop)

#
# Save this for future use.
fpts_df.to_csv('../tmp/01/player_points.csv', index=False)